# Exploratory Data and Domain Analysis

This notebook dedicated for exploring the domain and data available for better understanding of the problem, providing suitable solutions with the right methods.

Multipule datasets has been collected from deferent sources to be analyzed. the data consist of 4 main topics: 
- Accidents Severity.
- Traffic Accedints.
- RTA Global.
- Accidents Statisitics.

In [ ]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

## 1. Traffic Mortality and Policy Interventions

### EP Traffic Mortality and Policy Interventions Dataset

Source: Mendeley Data

Description:

A comprehensive dataset examining the relationship between traffic mortality rates and policy interventions (such as road safety regulations, awareness programs, or infrastructure improvements) in the Eastern Province.

Key Features: Mortality counts, intervention types, time period, geographic indicators, and possibly socio-economic factors.

Use: To explore the impact of government initiatives on traffic-related deaths.

In [ ]:
# xslx file
df = pd.read_excel("/content/EP-traffic-mortality-and-policy-interventions-dataset.xlsx")
df.head()

In [ ]:
# Load Excel file
excel_file_path = "/content/EP-traffic-mortality-and-policy-interventions-dataset.xlsx"

# Check the sheets
datafile = pd.ExcelFile(excel_file_path)
print(datafile.sheet_names)

In [ ]:
# Load the second sheet Raw Accident Mortality Data
df = pd.read_excel(excel_file_path, sheet_name=datafile.sheet_names[1])
df.head()

In [ ]:
# drop the first column and the last five columns
df = df.drop(df.columns[0], axis=1)
df = df.iloc[:, :-5]

# we drop the columns 4 and 5
df = df.drop(df.columns[[3, 4]], axis=1)

# And drop the first row
df = df.drop(df.index[0])
df.head()

In [ ]:
# we make the first row , the columns name
df.columns = df.iloc[0]
df = df.drop(df.index[0])
df.head()

In [ ]:
# remove the second row
df = df.drop(df.index[0])

# rename the NaN column>> "Age"
df = df.rename(columns={np.nan: 'Age'})

df.head()

In [ ]:
# the last row need to be removed
df = df.drop(df.index[-1])
df.tail()

##### Data Exploration

In [ ]:
# Shape and columns
print("Rows:", df.shape[0])
print("Columns:", df.shape[1])
print("Columns:", df.columns.tolist())

In [ ]:
# Check missing values
df.isna().sum()

In [ ]:
# Basic info and data types
df.info()

In [ ]:
# Summary statistics
df.describe(include='all')

In [ ]:
# duplicates
df.duplicated().sum()

In [ ]:
# check the duplicate rows that are true
df[df.duplicated()]

##### Data Cleaning

In [ ]:
# Rename columns
df.columns = df.columns.str.strip().str.replace(' ', '_').str.lower()
df.columns

In [ ]:
# rename death_date_(gregorian) > death_date_gregorian & death_date_(hijri) > death_date_hijri
df = df.rename(columns={'death_date_(gregorian)': 'death_date_gregorian', 'death_date_(hijri)': 'death_date_hijri'})
df.columns

In [ ]:
# Drop duplicates
df = df.drop_duplicates()
df.duplicated().sum()

In [ ]:
df.shape

12 row duplicated!

from 7351 >> 7339

In [ ]:
# handle mising values in hosiptal_code with "Unknown"
df['hospital_code'] = df['hospital_code'].fillna('Unknown')
df.isna().sum()

In [ ]:
# Convert dates to datetime
df['death_date_gregorian'] = pd.to_datetime(df['death_date_gregorian'], errors='coerce', dayfirst = True)

# Extract data
df['greg_year'] = df['death_date_gregorian'].dt.year
df['greg_month'] = df['death_date_gregorian'].dt.month
df['greg_day'] = df['death_date_gregorian'].dt.day

In [ ]:
# Extract data from hijri dates
df[['hijri_day', 'hijri_month', 'hijri_year']] = df['death_date_hijri'].str.split('/', expand=True)

In [ ]:
# change dtypes

num_cols = ['age', 'population', 'hospital_code', 'hijri_day', 'hijri_month', 'hijri_year', 'greg_year', 'greg_month', 'greg_day']

for col in num_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

cat_cols = ['place_of_death', 'gender', 'nationality', 'city', 'governorate']

for col in cat_cols:
    df[col] = df[col].astype('category')

df.info()

In [ ]:
# Hijri month names
hijri_months = {
    1: 'Muharram', 2: 'Safar', 3: 'Rabi al-Awwal', 4: 'Rabi al-Thani',
    5: 'Jumada al-Awwal', 6: 'Jumada al-Thani', 7: 'Rajab', 8: 'Shaaban',
    9: 'Ramadan', 10: 'Shawwal', 11: 'Dhul-Qadah', 12: 'Dhul-Hijjah'
}
df['hijri_month_name'] = df['hijri_month'].map(hijri_months)

# Gregorian month names
greg_months = {
    1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun',
    7:'Jul', 8:'Aug', 9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'
}
df['greg_month_name'] = df['greg_month'].map(greg_months)

In [ ]:
# adding age_group column
age_groups = {
    '0-14': (0, 14), '15-24': (15, 24), '25-34': (25, 34), '35-44': (35, 44),
    '45-54': (45, 54), '55-64': (55, 64), '65-74': (65, 74), '75-84': (75, 84),
    '85+': (85, float('inf')) # 85 to infinity
}
df['age_group'] = df['age'].apply(lambda x: next((k for k, v in age_groups.items() if v[0] <= x <= v[1]), None))

In [ ]:
# save the cleaned data
df.to_csv('Traffic_Accident_Mortality_Primary_cleaned.csv', index=False)

##### EDA

## 2. General Road Traffic Accident Patterns

### Road Traffic Accidents

Source: Kaggle

Description:

A global or multi-regional dataset containing records of individual traffic accidents. It includes features like accident type, severity, weather, road surface, time, and vehicle type.
Key Features: Accident date/time, location, cause, vehicle type, casualties.

Use: To perform general exploratory and comparative analysis of accident patterns and causes.

## 3. Historical Traffic Accident Statistics

### Traffic Accident Statistics (1437–1439 H)

Source: National Saudi Open Data Portal

Description:

Merged official records from three Hijri years (1437–1439 H), offering an authentic historical overview of traffic accident statistics across Saudi Arabia.

Key Features: Yearly total accidents, injuries, deaths, accident causes, and possibly province-level breakdowns.

Use: To analyze temporal trends and changes in accident frequency and severity over time.

## 4. Injuries and Deaths from Road Traffic Accidents

### Injuries and Deaths from Road Traffic Accidents (Last Ten Years)

Source: National Saudi Open Data Portal

Description:

A longitudinal dataset tracking injuries and fatalities from road traffic accidents over a decade, likely aggregated annually at the national or regional level.

Key Features: Year, total injuries, deaths, ratios, possible demographic splits.

Use: To study long-term national trends and compare them with policy periods or other datasets.